# Visual analysis of the results

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import numpy as np
from sklearn import metrics
from sklearn import calibration

plt.rcParams['savefig.dpi'] = 500
# set xticks fontsize
plt.rcParams['xtick.labelsize'] = 18
# set yticks fontsize
plt.rcParams['ytick.labelsize'] = 18
# set labels fontsize
plt.rcParams['axes.labelsize'] = 22
# set legend fontsize
plt.rcParams['legend.fontsize'] = 12
# set labels weight to bold
plt.rcParams['axes.labelweight'] = 'bold'
# set figure size
plt.rcParams['figure.figsize'] = (10, 10)

In [ ]:
experiment_dir = '../tests'

In [ ]:
test_name = experiment_dir.split('tests/')[1].replace('/', '-')

In [ ]:
model = 'LogisticRegression'

## Confusion Matrices

In [ ]:
tps, fps, tns, fns, roc_aucs = [], [], [], [], []
for run in sorted(os.listdir(experiment_dir)):
    run_dir = os.path.join(experiment_dir, run)
    try:
        with open(os.path.join(run_dir, 'results.json')) as f:
            results = json.load(f)
        roc_aucs += results[model]['roc_auc_score']
        tps += results[model]['tp']
        fps += results[model]['fp']
        tns += results[model]['tn']
        fns += results[model]['fn']
    except:
        pass

cm = np.array([[np.mean(tns), np.mean(fps)], [np.mean(fns), np.mean(tps)]])
std_cm = np.array([[np.std(tns), np.std(fps)], [np.std(fns), np.std(tps)]])

In [ ]:
if std_cm[0][0] == 0:
    group_counts = ['{0:0.2f}'.format(value) for value in cm.flatten()]
    percentages_cm = (cm.T / cm.sum(axis=1)).T
    group_percentages = ['{0:.2%}'.format(value) for value in percentages_cm.flatten()]
else:
    group_counts = ['{0:0.2f} ± {1:0.2f}'.format(value, std) for value, std in zip(cm.flatten(), std_cm.flatten())]
    percentages_cm = (cm.T / cm.sum(axis=1)).T
    # add percentages std
    percentages_std = (std_cm.T / cm.sum(axis=1)).T
    group_percentages = ['{0:.2%} ± {1:.2%}'.format(value, std) for value, std in zip(percentages_cm.flatten(), percentages_std.flatten())]
#     group_percentages = ['{0:.2%}'.format(value) for value in percentages_cm.flatten()]

labels = [f'{v1}\n({v2})' for v1, v2 in zip(group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)

plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=labels, fmt='', cmap='Blues', annot_kws={"size": 22})
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.savefig(f'../images/{test_name}-{model}-CM.png', bbox_inches='tight')
plt.savefig(f'../images/{test_name}-{model}-CM.svg', bbox_inches='tight')
plt.savefig(f'../images/{test_name}-{model}-CM.pdf', bbox_inches='tight')
plt.show()

## AUCROC curves

In [ ]:
models = ['LogisticRegression', 'RandomForestClassifier', 'XGBClassifier', 'MLPClassifier', 'AdaBoostClassifier', 'SVC']

In [ ]:
NUMBER_BINS = 10

In [ ]:
preds_values = {}
for model in models:
    mean_fpr = np.linspace(0, 1, 100)
    tpr_rates = []
    roc_scores = []
    prob_true_list = []
    prob_pred_list = []
    # plt.figure(figsize=(10, 10))
    for exp in os.listdir(experiment_dir):
        try:
            preds_path = os.path.join(experiment_dir, exp, 'predictions')
            models_preds = [f for f in os.listdir(preds_path) if model in f]
            for preds_file in models_preds:
                aux_path = os.path.join(preds_path, preds_file)
                preds_df = pd.read_csv(aux_path)
                # roc metric
                roc_score = metrics.roc_auc_score(preds_df.y_true, preds_df.y_proba_1)
                roc_scores.append(roc_score)
                # roc curve
                fpr_proba, tpr_proba, threshold_proba = metrics.roc_curve(preds_df.y_true, preds_df.y_proba_1)
                interp_tpr = np.interp(mean_fpr, fpr_proba, tpr_proba)
                interp_tpr[0] = 0.0
                tpr_rates.append(interp_tpr)
                # calibration tool
                prob_true, prob_pred = calibration.calibration_curve(preds_df.y_true, preds_df.y_proba_1, n_bins=NUMBER_BINS)
                # complete the list with nan if the length is less than NUMBER_BINS
                if len(prob_true) < NUMBER_BINS:
                    prob_true = np.concatenate((prob_true, np.full(NUMBER_BINS - len(prob_true), np.nan)))
                    prob_pred = np.concatenate((prob_pred, np.full(NUMBER_BINS - len(prob_pred), np.nan)))
                prob_true_list.append(prob_true)
                prob_pred_list.append(prob_pred)
        except:
            pass

    try:
        mean_tpr = np.mean(tpr_rates, axis=0)
        mean_tpr[-1] = 1.0

        mean_prob_true = np.nanmean(prob_true_list, axis=0)
        mean_prob_pred = np.nanmean(prob_pred_list, axis=0)

        preds_values[model] = {
            'fpr' : mean_fpr,
            'tpr' : mean_tpr,
            'mean' : np.mean(roc_scores),
            'std' : np.std(roc_scores),
            'prob_true' : mean_prob_true,
            'prob_pred' : mean_prob_pred
        }
    except:
        pass

In [ ]:
# sort preds_values by mean
preds_values = dict(sorted(preds_values.items(), key=lambda item: item[1]['mean'], reverse=True))

### ROC CURVE ALL models

In [ ]:
ticks = np.linspace(0, 1, 11)
ticks

In [ ]:
plt.figure()
sns.set_style("whitegrid")
for model, result in preds_values.items():
    lw = 3
    plt.plot(
        result['fpr'], 
        result['tpr'], 
        label=f'Mean ROC (AUC= {result["mean"].round(3)} $\pm$ {result["std"].round(3)}) - {model}',
        lw=lw)
plt.plot([0, 1], [0, 1], linewidth=2, linestyle='dashed', color = 'g', label='Random Classifier')
    
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.xticks(ticks)
plt.yticks(ticks)
plt.grid(linestyle='--', alpha=0.7, linewidth=1)
plt.savefig(f'../images/{test_name}-ROC-models.png', bbox_inches='tight')
plt.savefig(f'../images/{test_name}-ROC-models.svg', bbox_inches='tight')
plt.savefig(f'../images/{test_name}-ROC-models.pdf', bbox_inches='tight')
plt.show()

In [ ]:
plt.figure()
sns.set_style("whitegrid")
for model, result in preds_values.items():
    lw = 3
    if model == 'LogisticRegression':
        plt.plot(
            result['fpr'], 
            result['tpr'], 
            label=f'Mean ROC (AUC= {result["mean"].round(3)} $\pm$ {result["std"].round(3)}) - {model}',
            lw=lw)
plt.plot([0, 1], [0, 1], linewidth=2, linestyle='dashed', color = 'g', label='Random Classifier')
    
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.xticks(ticks)
plt.yticks(ticks)
plt.grid(linestyle='--', alpha=0.7, linewidth=1)
plt.savefig(f'../images/{test_name}-ROC-lr.png', bbox_inches='tight')
plt.savefig(f'../images/{test_name}-ROC-lr.svg', bbox_inches='tight')
plt.savefig(f'../images/{test_name}-ROC-lr.pdf', bbox_inches='tight')
plt.show()

### Calibration Curve

In [ ]:
plt.figure()
sns.set_style("whitegrid")
plt.plot([0, 1], [0, 1], linewidth=2, linestyle='dashed', color='black', label='Perfectly Calibrated')
for model, result in preds_values.items():
    lw = 3
    plt.plot(
        result['prob_pred'], 
        result['prob_true'], 
        label=f'{model}',
        lw=lw, marker='o')
    
plt.xlabel('Predicted Probability')
plt.ylabel('True Probability')
# save legend outside the plot
plt.legend()
plt.xticks(ticks)
plt.yticks(ticks)
plt.grid(linestyle='--', alpha=0.7, linewidth=1)
plt.savefig(f'../images/{test_name}-calibration-models.png', bbox_inches='tight')
plt.savefig(f'../images/{test_name}-calibration-models.svg', bbox_inches='tight')
plt.savefig(f'../images/{test_name}-calibration-models.pdf', bbox_inches='tight')
plt.show()

In [ ]:
plt.figure()
sns.set_style("whitegrid")
plt.plot([0, 1], [0, 1], linewidth=2, linestyle='dashed', color='black', label='Perfectly Calibrated')
for model, result in preds_values.items():
    lw = 3
    if model == 'LogisticRegression':
        plt.plot(
            result['prob_pred'], 
            result['prob_true'], 
            label=f'{model}',
            lw=lw, marker='o')
    
plt.xlabel('Predicted Probability')
plt.ylabel('True Probability')
# save legend outside the plot
plt.legend()
plt.xticks(ticks)
plt.yticks(ticks)
plt.grid(linestyle='--', alpha=0.7, linewidth=1)
plt.savefig(f'../images/{test_name}-calibration-lr.png', bbox_inches='tight')
plt.savefig(f'../images/{test_name}-calibration-lr.svg', bbox_inches='tight')
plt.savefig(f'../images/{test_name}-calibration-lr.pdf', bbox_inches='tight')
plt.show()